## Метод максимального правдоподобия. Хи-квадрат.

Необходимо сгенерировать выборку объёмом 100 элементов для нормального распределения $N(x,0,1)$. По сгенерированной выборке оценить параметры $\mu$ и $\sigma$ нормального закона методом максимального правдоподобия. В качестве основной гипотезы $H_0$ будем считать, что сгенерированное распределение имеет вид $N(x, \hat \mu, \hat \omega)$. Проверить основную гипотезу, используя критерий согласия $\chi^{2}$. В качестве уровня значимости взять $\alpha = 0.05$. Привести таблицу вычислений $\chi^{2}$. Исследовать точность (чувствительность) критерия $\chi^{2} - $ сгенерировать выборки равномерного распределения и распределения Лапласа малого объема (например, 20 элементов). Проверить их на нормальность.

Подключаем все библиотеки

In [1]:
import numpy as np
from scipy.stats import laplace, uniform
from tabulate import tabulate
import scipy.stats as stats
import math as m

Задаём все необходимые константы

In [2]:
sizes = [20, 100]
alpha = 0.05
p = 1 - alpha

Функция, вычисляющая значение $k$ в зависимости от объема выборки

In [3]:
def get_k(size):
    return m.ceil(1.72 * (size) ** (1/3))

Функция, находящая $\mu~и~\sigma$, а также число интервалов. С помощью встроенной функции в библиотеку scipy считает $\chi^{2}$

In [4]:
def calculate(distribution):
    mu = np.mean(distribution)
    sigma = np.std(distribution)

    print('mu = ' + str(np.around(mu, decimals=2)))
    print('sigma = ' + str(np.around(sigma, decimals=2)))
    
    limits = np.linspace(-1.1, 1.1, num=k - 1)
    chi_2 = stats.chi2.ppf(p, k-1)
    print('chi_2 = ' + str(chi_2))
    return limits

Функция, которая на соответсвующих интервалах вычисляет частоты попадания выборочных элементов и их вероятности

In [9]:
def get_n_and_p(distribution, limits, size):
    p_list = np.array([])
    n_list = np.array([])
    
    for i in range(-1, len(limits)):
        if i != -1:
            prev_cdf_val = stats.norm.cdf(limits[i])
        else:
            prev_cdf_val = 0
        if i != len(limits) - 1:
            cur_cdf_val = stats.norm.cdf(limits[i+1])
        else: 
            cur_cdf_val = 1 
        p_list = np.append(p_list, cur_cdf_val - prev_cdf_val)
        if i == -1:
            n_list = np.append(n_list, len(distribution[distribution <= limits[0]]))
        elif i == len(limits) - 1:
            n_list = np.append(n_list, len(distribution[distribution >= limits[-1]]))
        else:
            n_list = np.append(n_list, len(distribution[(distribution <= limits[i + 1]) & (distribution >= limits[i])]))

    result = np.divide(np.multiply((n_list - size * p_list), (n_list - size * p_list)), p_list * size)
    return n_list, p_list, result

Функция, отрисовывающая таблицу в формате LaTeX

In [11]:
def create_table(n_list, p_list, result, size):
    
    cols = ["i", "limits", "n_i", "p_i", "np_i", "n_i - np_i", "/frac{(n_i-np_i)^2}{np_i}"]
    rows = []
    for i in range(0, len(n_list)):
        if i == 0:
            boarders = ['-inf', np.around(limits[0], decimals=2)]
        elif i == len(n_list) - 1:
            boarders = [np.around(limits[-1], decimals=2), 'inf']
        else:
            boarders = [np.around(limits[i - 1], decimals=2), np.around(limits[i], decimals=2)]

        rows.append([i + 1, boarders, n_list[i], np.around(p_list[i], decimals=4), np.around(p_list[i] * size, decimals=2),
                 np.around(n_list[i] - size * p_list[i], decimals=2), np.around(result[i], decimals=2)])

    rows.append([len(n_list), "-", np.sum(n_list), np.around(np.sum(p_list), decimals=4),
             np.around(np.sum(p_list * size), decimals=2),
             -np.around(np.sum(n_list - size * p_list), decimals=2),
             np.around(np.sum(result), decimals=2)])
    print(tabulate(rows, cols, tablefmt="latex"))

Строим нормальное распределение

In [12]:
k = get_k(sizes[1])
distribution = np.random.normal(0, 1, size=sizes[1])
limits = calculate(distribution)
n_list, p_list, result = get_n_and_p(distribution, limits, sizes[1])
create_table(n_list, p_list, result, sizes[1])

mu = -0.05
sigma = 0.95
chi_2 = 14.067140449340169
\begin{tabular}{rlrrrrr}
\hline
   i & limits         &   n\_i &    p\_i &   np\_i &   n\_i - np\_i &   /frac\{(n\_i-np\_i)\^{}2\}\{np\_i\} \\
\hline
   1 & ['-inf', -1.1] &    15 & 0.1357 &  13.57 &         1.43 &                        0.15 \\
   2 & [-1.1, -0.73]  &     9 & 0.096  &   9.6  &        -0.6  &                        0.04 \\
   3 & [-0.73, -0.37] &    13 & 0.1253 &  12.53 &         0.47 &                        0.02 \\
   4 & [-0.37, 0.0]   &    20 & 0.1431 &  14.31 &         5.69 &                        2.27 \\
   5 & [0.0, 0.37]    &     9 & 0.1431 &  14.31 &        -5.31 &                        1.97 \\
   6 & [0.37, 0.73]   &     8 & 0.1253 &  12.53 &        -4.53 &                        1.64 \\
   7 & [0.73, 1.1]    &    14 & 0.096  &   9.6  &         4.4  &                        2.02 \\
   8 & [1.1, 'inf']   &    12 & 0.1357 &  13.57 &        -1.57 &                        0.18 \\
   8 & -              &   100 &

Проделаем аналогичные выкладки для распределения Лапласа для 20 элементов

In [13]:
k = get_k(sizes[0])
distribution = laplace.rvs(size=sizes[0], scale=1 / m.sqrt(2), loc=0)
limits = calculate(distribution)
n_list, p_list, result = get_n_and_p(distribution, limits, sizes[0])
create_table(n_list, p_list, result, sizes[0])

mu = -0.14
sigma = 1.01
chi_2 = 9.487729036781154
\begin{tabular}{rlrrrrr}
\hline
   i & limits         &   n\_i &    p\_i &   np\_i &   n\_i - np\_i &   /frac\{(n\_i-np\_i)\^{}2\}\{np\_i\} \\
\hline
   1 & ['-inf', -1.1] &     3 & 0.1357 &   2.71 &         0.29 &                        0.03 \\
   2 & [-1.1, -0.37]  &     4 & 0.2213 &   4.43 &        -0.43 &                        0.04 \\
   3 & [-0.37, 0.37]  &     8 & 0.2861 &   5.72 &         2.28 &                        0.91 \\
   4 & [0.37, 1.1]    &     3 & 0.2213 &   4.43 &        -1.43 &                        0.46 \\
   5 & [1.1, 'inf']   &     2 & 0.1357 &   2.71 &        -0.71 &                        0.19 \\
   5 & -              &    20 & 1      &  20    &        -0    &                        1.62 \\
\hline
\end{tabular}


Проделаем аналогичные выкладки для равномерного распределения с 20 элементами

In [15]:
k = get_k(sizes[0])
distribution = uniform.rvs(size=sizes[0], loc=-m.sqrt(3), scale=2 * m.sqrt(3))
limits = calculate(distribution)
n_list, p_list, result = get_n_and_p(distribution, limits, sizes[0])
create_table(n_list, p_list, result, sizes[0])

mu = -0.23
sigma = 1.03
chi_2 = 9.487729036781154
\begin{tabular}{rlrrrrr}
\hline
   i & limits         &   n\_i &    p\_i &   np\_i &   n\_i - np\_i &   /frac\{(n\_i-np\_i)\^{}2\}\{np\_i\} \\
\hline
   1 & ['-inf', -1.1] &     5 & 0.1357 &   2.71 &         2.29 &                        1.93 \\
   2 & [-1.1, -0.37]  &     5 & 0.2213 &   4.43 &         0.57 &                        0.07 \\
   3 & [-0.37, 0.37]  &     4 & 0.2861 &   5.72 &        -1.72 &                        0.52 \\
   4 & [0.37, 1.1]    &     4 & 0.2213 &   4.43 &        -0.43 &                        0.04 \\
   5 & [1.1, 'inf']   &     2 & 0.1357 &   2.71 &        -0.71 &                        0.19 \\
   5 & -              &    20 & 1      &  20    &        -0    &                        2.75 \\
\hline
\end{tabular}
